In [17]:
import numpy as np
from numpy import absolute, mean, std
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline

import random

random.seed(10)

# https://machinelearningmastery.com/rfe-feature-selection-in-python/

In [18]:
#work_p = "C:/Users/claire.smid/Documents/Main_STUDY/Model-based_learning_mainstudy/"

work_p = "C:/Users/claire.smid/Documents/DCP/Writing/Thesis/Chapter2/"

# Including all decision making, EFs and mental health:
#dataset = pd.read_csv(work_p+"All_DMs_EFs_MHs_Nov21.csv", sep=',') 

# Including all decision making and EFs
#dataset = pd.read_csv(work_p+"All_DMs_EFs_Nov21.csv", sep=',') 

# Including EFs and MB only
#dataset = pd.read_csv(work_p+"MB_EFs_Only_Nov21.csv", sep=',') 
#dataset = pd.read_csv(work_p+"MB_EF_Detail_Imputed_Dec21.csv", sep=',') 
#dataset = pd.read_csv(work_p+"MB_EF_Detail_Imputed_REDUCED_Dec21.csv", sep=',') 
#dataset = pd.read_csv(work_p+"MBMF_EF_Detail_Imp_6Jun2022.csv", sep=',') 
dataset = pd.read_csv(work_p+"MBMF_EF_Detail_Imp_7Jun2022.csv", sep=',') 

pd.options.display.max_columns = 100

dataset

,ID,Session,School,Gender,Age_Frac_Imp,SES_inv_z,w_P6,w_diff,it_P6,lr_P6,eg_P6,st_P6,repst_P6,it,lr,eg,st,repst,Avg_Pts,Avg_Pts_lo,Avg_Pts_hi,T_Vocab,T_Matrix,Corsi_WM_Span,AXCPT_t,CogFlex_SwitchRT_t,SSRT,FlankerSwitch_t,FlankerInhib_t,Stroop_t,axcpt_PBI_t0,AXCPT_CorrRT_Shift_Index,AY_RCS,BX_RCS,Flank_Switch_RCS_Diff,Flank_InCon_RCS_Diff,Nback_1_RCS,Nback_2_RCS,CF_Switch_RCS_Diff,CF_Mix_RCS_Diff,SSRT_SSD_z,Corsi_CorrRT_log
0,1,0,Trevor Roberts School,M,12.444444,0.226732,0.425788,0.121567,0.619787,0.437227,0.419932,-0.177439,-0.709067,0.625122,0.442486,0.420558,-0.177359,-0.704026,0.013173,-0.002222,0.029551,62,66,7,-0.696260,176.281070,-0.798678,0.035289,-0.303618,-0.818045,-0.696260,0.260224,1.785449,0.927080,-0.308600,-0.498360,0.241960,0.114083,-0.405414,-0.699165,0.316186,-7.521572
1,4,0,Trevor Roberts School,F,11.838889,-0.587075,0.528114,-0.142475,0.804932,0.698279,0.486952,-0.435025,-0.862134,0.812849,0.685928,0.486859,-0.426516,-0.851632,0.024752,0.004831,0.041414,55,50,5,0.431760,93.596497,0.028908,0.242714,0.563637,-0.013837,0.431760,0.371609,0.985358,1.401536,-0.264254,-0.086407,0.272975,0.123974,-0.720982,-0.018518,-0.640517,-7.302517
2,5,0,Trevor Roberts School,M,7.611111,1.040539,0.770192,0.132597,0.880334,0.755154,0.530449,-0.589862,-0.083494,0.846292,0.747178,0.556262,-0.601976,-0.080824,0.043889,0.037037,0.052972,66,55,4,-0.433764,123.828610,-0.380691,0.525759,-0.140001,-0.205424,-0.433764,0.056031,0.990982,0.373066,0.011101,-0.140620,0.117747,0.174845,0.054496,-0.171262,1.903905,-7.973345
3,8,0,Trevor Roberts School,M,6.747222,0.226732,0.582383,0.028154,0.651551,0.764537,0.577141,0.588800,0.396509,0.652178,0.783163,0.582775,0.589636,0.395179,0.013740,-0.022676,0.054293,64,61,4,-0.614938,800.391720,-1.489266,-0.041334,-0.248403,1.083356,-0.063924,0.029162,0.562811,0.437377,-0.314666,-0.261631,0.102242,0.228547,0.040507,0.057369,-0.071924,-8.065688
4,15,0,Trevor Roberts School,M,10.600000,0.226732,0.448529,-0.036160,0.553742,0.489282,0.530107,-0.303803,-1.629661,0.553467,0.487266,0.528799,-0.305209,-1.630474,-0.014891,0.016414,-0.040881,55,62,6,-0.597904,195.210570,0.016327,-0.679340,-0.246747,0.085989,-0.597904,0.121451,0.461337,0.558434,-0.714397,-0.277322,0.216780,0.260057,-0.072714,0.025455,-1.590435,-7.684010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,193,0,Broadfields,M,10.486111,-2.214688,0.422117,0.117357,0.433004,0.519166,0.491737,0.173996,-0.269121,0.436899,0.517092,0.471755,0.175653,-0.262585,0.098073,0.071605,0.120545,41,56,6,-0.059368,182.111080,0.200856,-0.572747,0.212403,0.187235,-0.059368,0.219358,1.589251,2.482398,-0.671223,-0.117612,0.174274,0.156276,-0.509668,0.034773,1.374665,-7.365308
65,195,0,Merton Court,M,10.372222,-0.587075,0.610913,-0.218845,0.445750,0.349609,0.394229,-0.074491,-0.851411,0.453946,0.367773,0.398445,-0.070527,-0.840291,-0.017052,-0.065302,0.045455,58,64,6,-0.064548,349.235350,-0.707812,-0.136637,0.689489,-0.980713,-0.064548,0.089907,0.913796,1.094341,-0.225866,0.051139,0.217520,0.150937,-0.149499,-0.266687,-1.236251,-7.367070
66,196,0,Orley Farm,M,8.608333,0.498001,0.718295,0.107204,1.381509,0.868366,0.658000,0.382585,-0.281852,1.387507,0.884099,0.678044,0.386831,-0.277990,0.057756,0.049645,0.064815,64,52,5,-0.435616,437.055540,0.909617,0.208257,-0.148340,-0.622866,0.600552,0.008661,1.036768,0.302673,-0.125693,-0.081658,0.209424,0.273568,0.040507,-0.571175,0.139772,-7.444318
67,202,0,Newland House,F,11.236111,1.040539,0.893768,0.043038,1.296684,0.839617,0.514458,0.758670,-0.065990,1.165809,0.808938,0.521772,0.873589,-0.060433,0.044004,0.045351,0.042735,63,54,7,-0.438972,82.833313,-0.562425,-0.814565,0.551108,-0.565334,-0.438972,-0.026887,0.818956,0.818888,-0.770009,0.112121,0.140716,0.135013,-0.511400,0.012360,-1.571437,-7.552011


In [19]:
# one hot encoding
dataset.Gender.replace(to_replace=['M','F'],value=[1,0],inplace=True)

In [20]:
# # 6 jun dataset
# X = dataset.iloc[:, np.r_[14:34]]
# X_cols = X
# y = dataset.iloc[:,8]

# 7 jun dataset
X = dataset.iloc[:, np.r_[21:42]]
X_cols = X
y = dataset.iloc[:,7]

## LOOCV with all features

In [21]:
# find number of samples
n = X.shape[0]
# create loocv procedure
cv = LeaveOneOut()
# create model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(X)
regressor = LinearRegression()
# evaluate model
scores = cross_val_score(regressor, X_poly, y, scoring='neg_mean_squared_error', cv=n)
# force positive
#scores = absolute(scores)

# report performance
print('MSE: %.3f (%.3f)' % (mean(scores), std(scores)))


MSE: -0.516 (1.273)


## LOOCV with the best predicting features

In [22]:
# explore the algorithm wrapped by RFE
# automatically select the number of features for RFE

# create pipeline
rfe = RFECV(estimator=LinearRegression(),min_features_to_select=2)
model = LinearRegression(fit_intercept=True,normalize=True)
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# find number of samples
n = X.shape[0]
poly_reg = PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(X)
# create loocv procedure
cv = LeaveOneOut()
# evaluate model
scores = cross_val_score(pipeline, X_poly, y, scoring='neg_mean_squared_error', cv=n)
# report performance
print('MSE: %.3f (%.3f)' % (mean(scores), std(scores)))

MSE: -0.045 (0.060)


### Predicting the best results

In [15]:
# find out which predictors
rfe.fit(X,y)
data_top = X_cols.columns
# summarise all features
for i in range(X.shape[1]):
    print('Selected %s,\t Rank: %.1f, \tName: %s' % (rfe.support_[i], rfe.ranking_[i],data_top[i]))

Selected False,	 Rank: 15.0, 	Name: T_Vocab
Selected False,	 Rank: 17.0, 	Name: T_Matrix
Selected False,	 Rank: 16.0, 	Name: Corsi_WM_Span
Selected False,	 Rank: 10.0, 	Name: SSRT
Selected False,	 Rank: 3.0, 	Name: FlankerSwitch_t
Selected True,	 Rank: 1.0, 	Name: FlankerInhib_t
Selected False,	 Rank: 2.0, 	Name: Stroop_t
Selected False,	 Rank: 4.0, 	Name: axcpt_PBI_t0
Selected False,	 Rank: 18.0, 	Name: Higher_Acc
Selected False,	 Rank: 19.0, 	Name: CorrRT_2SD
Selected False,	 Rank: 11.0, 	Name: AXCPT_CorrRT_Shift_Index
Selected False,	 Rank: 14.0, 	Name: AY_RCS
Selected False,	 Rank: 12.0, 	Name: BX_RCS
Selected False,	 Rank: 6.0, 	Name: Flank_Switch_RCS_Diff
Selected False,	 Rank: 13.0, 	Name: Flank_InCon_RCS_Diff
Selected True,	 Rank: 1.0, 	Name: Nback_1_RCS
Selected False,	 Rank: 7.0, 	Name: Nback_2_RCS
Selected False,	 Rank: 8.0, 	Name: CF_Switch_RCS_Diff
Selected False,	 Rank: 9.0, 	Name: CF_Mix_RCS_Diff
Selected False,	 Rank: 5.0, 	Name: SSRT_SSD_z


# Use permutation testing to find the best predicting features

In [16]:
from sklearn.model_selection import RepeatedKFold
import eli5
from eli5.sklearn import PermutationImportance
from eli5 import show_prediction, show_weights

Xfeature_names = X_cols.columns.tolist()

# create pipeline
regressor = LinearRegression()

cv = RepeatedKFold(n_splits=6, n_repeats=3, random_state=1)
perm = PermutationImportance(regressor, cv = cv)
perm.fit(X_poly,y)

show_weights(perm, feature_names = X_cols.columns.tolist())
#show_prediction(perm, X.iloc[4], feature_names = X.columns.tolist())

C:\Users\claire.smid\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classifier=False as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


ValueError: feature_names has a wrong length: expected=231, got=20

## Hyperparameter tuning

In [ ]:
regressor.get_params().keys()

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# make the model manually with the predictors
#X = dataset[["Higher_Acc","SSRT_mean_ssd"]]
# for w
X = dataset[["ssrt_t0","Stroop_InCon_IES_Diff"]]

# find number of samples
n = X.shape[0]
# create loocv procedure
cv = LeaveOneOut()
# create model
regressor = LinearRegression()
# define search space
space = dict()
#space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
#space['alpha'] =  [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
space['fit_intercept'] = [True, False]
space['normalize'] = [True, False]
# define search
search = GridSearchCV(regressor,space,scoring='neg_mean_squared_error',cv=cv, n_jobs = -1)
# execute search
result = search.fit(X_poly,y)

# summarize result
print('Best MSE Score: %s' % result.best_score_)
print('Best Hyperparameters %s' % result.best_params_ )



## K-fold cross validation approach with RFE

In [ ]:
# explore the algorithm wrapped by RFE
# automatically select the number of features for RFE
from sklearn.model_selection import RepeatedKFold



poly_reg = PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(X)

# create pipeline
regressor = LinearRegression()
# evaluate model
cv = RepeatedKFold(n_splits=6, n_repeats=3, random_state=1)
n_scores = cross_val_score(regressor, X_poly, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
r2_scores = cross_val_score(regressor, X_poly, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
explained_var = cross_val_score(regressor, X, y, scoring='explained_variance', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('MSE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print('r2: %.3f (%.3f)' % (mean(r2_scores), std(r2_scores)))
print('exp var: %.3f' % mean(explained_var))

## Perumutation testing

In [ ]:
from sklearn.model_selection import RepeatedKFold
import eli5
from eli5.sklearn import PermutationImportance
from eli5 import show_prediction, show_weights

# make the model manually with the predictors
# for metacontrol
X = dataset[["ssrt_t0","flankerinh_t0","Stroop_InCon_IES_Diff","Nback_1_2_RCS_Diff"]]
# for w
#X = dataset[["ssrt_t0","Stroop_InCon_IES_Diff"]]
# for lr
#X = dataset[["ssrt_t0","axcpt_PBI_t0","Stroop_InCon_IES_Diff","SSRT_mean_ssd"]]

Xfeature_names = X.columns


# create pipeline
regressor = LinearRegression(fit_intercept=False,normalize=True)

cv = RepeatedKFold(n_splits=6, n_repeats=3, random_state=1)
perm = PermutationImportance(regressor, cv = cv)
perm.fit(X,y)

show_weights(perm, feature_names = X.columns.tolist())
#show_prediction(perm, show_feature_values=True)

## Repeat with PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 0.90)

# from column 8 onwards no more model parameters, or maybe just include all?
# now including only the EFs for the first approach, and splitting factors for the later approach
X = dataset.iloc[:,12:]
y = dataset.iloc[:,1]

pca.fit(X)
pc_X = pca.transform(X)

poly_reg = PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(pc_X)

# find number of sam`aples
n = pc_X.shape[0]
# create loocv procedure
cv = LeaveOneOut()
# create model
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
# evaluate model
scores = cross_val_score(regressor, pc_X, y, scoring='neg_mean_squared_error', cv=n)
# force positive
#scores = absolute(scores)

# report performance
print('MSE: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
# explore the algorithm wrapped by RFE
# automatically select the number of features for RFE
from sklearn.linear_model import LinearRegression

# create pipeline
rfe = RFECV(estimator=LinearRegression(),min_features_to_select=1)
model = LinearRegression()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# find number of samples
n = pc_X.shape[0]
# create loocv procedure
cv = LeaveOneOut()
# evaluate model
scores = cross_val_score(pipeline, pc_X, y, scoring='neg_mean_squared_error', cv=n, n_jobs = -1)

# report performance
print('MSE: %.3f (%.3f)' % (mean(scores), std(scores)))


In [ ]:
# find out which predictors
rfe.fit(pc_X,y)
data_top = ['MemSpeed_negBXRCS','posBXRCS_MemSpeed','PBI_InConIES',
                                                   'negSwitchIES_negPBI','DP2_negCFSwitchIES',
                                                   'negCFSwitchIES_InconIES','negInConIES_AYRCS']
# summarise all features
for i in range(pc_X.shape[1]):
    print('Selected %s,\t Rank: %.1f, \tName: %s' % (rfe.support_[i], rfe.ranking_[i],data_top[i]))

In [ ]:
# explore the algorithm wrapped by RFE
# automatically select the number of features for RFE
from sklearn.model_selection import RepeatedKFold

# create pipeline
rfe = RFECV(estimator=LinearRegression())
model = LinearRegression()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, pc_X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
r2_scores = cross_val_score(pipeline, pc_X, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
explained_var = cross_val_score(regressor, X, y, scoring='explained_variance', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('MSE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print('r2: %.3f (%.3f)' % (mean(r2_scores), std(r2_scores)))
print('exp var: %.3f' % mean(explained_var))